In [1]:
import os

In [2]:
%pwd

'd:\\ML\\text-summarizer-project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\ML\\text-summarizer-project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    load_data_file: Path
    unzip_dir: Path


In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import *

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            load_data_file=config.load_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.load_data_file):
            logger.info("Downloading dataset using Hugging Face load_dataset")

            dataset = load_dataset(
                self.config.dataset_name,  # e.g. "knkarthick/samsum"
            )

            os.makedirs(self.config.load_data_file, exist_ok=True)

            for split, ds in dataset.items():
                output_path = os.path.join(
                    self.config.load_data_file, f"{split}.json"
                )
                ds.to_json(output_path)
                logger.info(f"Saved {split} split to {output_path}")

        else:
            logger.info(
                f"Dataset already exists at {self.config.load_data_file} "
                f"with size: {get_size(Path(self.config.load_data_file))}"
            )

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.load_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-05 18:28:19,068]: INFO: common:  yaml file: config\config.yaml loaded successfully
[2026-01-05 18:28:19,070]: INFO: common:  yaml file: params.yaml loaded successfully
[2026-01-05 18:28:19,070]: INFO: common:  created directory at: artifacts
[2026-01-05 18:28:19,074]: INFO: common:  created directory at: artifacts/data_ingestion
[2026-01-05 18:28:19,764]: INFO: 3673121015:  artifacts/data_ingestion/data.zip download! with following info: 
Content-Type: text/html; charset=utf-8
Content-Length: 431912
Connection: close
Date: Mon, 05 Jan 2026 17:28:19 GMT
ETag: W/"69728-MMivypaTyRPJzgJNG5xGfyujDUE"
X-Powered-By: huggingface-moon
X-Request-Id: Root=1-695bf4b2-13b822e4753da9bb33df4fcd
RateLimit: "pages";r=99;t=120
RateLimit-Policy: "fixed window";"pages";q=100;w=300
cross-origin-opener-policy: same-origin
Referrer-Policy: strict-origin-when-cross-origin
x-frame-options: DENY
X-Cache: Miss from cloudfront
Via: 1.1 09dddedbac44fa07d4af5f638358fa8a.cloudfront.net (CloudFront)
X-Amz-Cf

BadZipFile: File is not a zip file

In [10]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("knkarthick/samsum")
ds

d:\ML\text-summarizer-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\ML\text-summarizer-project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\baska\.cache\huggingface\hub\datasets--knkarthick--samsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate develo

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14731
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})